# Chess Neural Network

Building a neural network that learns to play chess based on my gameplay data.

## Goal
Train a move prediction model to create an AI that plays like me.




In [ ]:
# Importing necessary libraries
import pandas as pd
import ast
import torch
import torch.nn as nn

## Step 1: Load Chess Data

Load cleaned data from csv file


In [2]:
df = pd.read_csv("/Users/riteshbhandari/Documents/GitHub/Chess-engine/bot/data-analysis/cleaned_data.csv")
df.head()

,game_id,moves,num_moves,first_move
0,123118274906,"[('white', 'e4', True), ('black', 'e6', False)...",90,e4
1,123118510404,"[('white', 'd4', False), ('black', 'c5', True)...",141,d4
2,123118790014,"[('white', 'e4', False), ('black', 'e5', True)...",46,e4
3,123158939328,"[('white', 'e4', False), ('black', 'd5', True)...",88,e4
4,123160166430,"[('white', 'e4', True), ('black', 'e5', False)...",110,e4


# Step 2: Pre-processing 
Encoding the features to be used 

In [ ]:
# convert string representation of list to actual list
df["moves"] = df["moves"].apply(ast.literal_eval)

# saving all the moves to single list to be encoded 
every_move = []
for game in df["moves"]:
    for move in game:
        every_move.append(move[1])

every_move

['e4',
 'e6',
 'd4',
 'Qh4',
 'Nc3',
 'f5',
 'Nf3',
 'Qe7',
 'e5',
 'Qb4',
 'Bd2',
 'Qxb2',
 'Rb1',
 'Qa3',
 'Nb5',
 'Qxa2',
 'Nxc7+',
 'Kd8',
 'Nxa8',
 'Ba3',
 'Ra1',
 'Qb2',
 'Bg5+',
 'Ne7',
 'Bxe7+',
 'Bxe7',
 'Rxa7',
 'Qc3+',
 'Nd2',
 'Nc6',
 'Ra1',
 'Nxd4',
 'Nb6',
 'Nxc2+',
 'Ke2',
 'Nxa1',
 'Nxc8',
 'Qxe5+',
 'Kf3',
 'Kxc8',
 'Qc1+',
 'Bc5',
 'Bb5',
 'Qd5+',
 'Ke2',
 'Nb3',
 'Nxb3',
 'Qe4+',
 'Kf1',
 'b6',
 'Nxc5',
 'bxc5',
 'Qxc5+',
 'Kd8',
 'Qd6',
 'Qb1+',
 'Ke2',
 'Qxb5+',
 'Kf3',
 'Re8',
 'Rd1',
 'Re7',
 'Kg3',
 'Ke8',
 'f4',
 'Qb3+',
 'Rd3',
 'Qb1',
 'Rd1',
 'Qb3+',
 'Rd3',
 'Qb1',
 'Kf2',
 'Qb2+',
 'Rd2',
 'Qf6',
 'g3',
 'e5',
 'Qb8+',
 'Kf7',
 'Qb3+',
 'Kg6',
 'Qf3',
 'e4',
 'Qg2',
 'e3+',
 'Ke2',
 'exd2+',
 'Kxd2',
 'Qb2+',
 'd4',
 'c5',
 'd5',
 'e5',
 'c4',
 'Qa5+',
 'Nc3',
 'Nf6',
 'd6',
 'Qb6',
 'Nf3',
 'Nc6',
 'b3',
 'Nb4',
 'Ba3',
 'Bxd6',
 'e3',
 'Bb8',
 'Bxb4',
 'cxb4',
 'Nd5',
 'Nxd5',
 'Qxd5',
 'O-O',
 'Nxe5',
 'Qe6',
 'Qxe6',
 'dxe6',
 'Nf3',
 'e5',
 'Be2',
 'a

In [27]:
# getting all the unique moves
unique_moves = set(every_move)  # just the unique moves
print("Number of different moves:", len(unique_moves))
print()

# give move a number
move_to_number = {}

# turning integer back to moves (for future use)
number_to_move= {}

for i, move in enumerate(unique_moves):
    move_to_number[move] = i
    number_to_move[i] = move

# turning all the numbers into integers
number_moves = []
for move in every_move:
    number_moves.append(move_to_number[move])

# first 10 moves
print("First 10 moves as numbers: ")
print(number_moves[:10])
print()

# first 10 original moves
print("First 10 original moves: ")
print(every_move[:10]) 

Number of different moves: 1927

First 10 moves as numbers: 
[265, 1191, 1198, 127, 1118, 1078, 455, 213, 1706, 1101]

First 10 original moves: 
['e4', 'e6', 'd4', 'Qh4', 'Nc3', 'f5', 'Nf3', 'Qe7', 'e5', 'Qb4']


In [28]:
# turn the teoriat moves to integers ( 1  = (True), 0 = (False) )
teoriat_moves = []
for game in df["moves"]:
    for move in game:
        if move[2] ==True:
            teoriat_moves.append(1)
        else: # False 
            teoriat_moves.append(0)
# first 10 teoriat moves
print("First 10 teoriat moves: ")
print(teoriat_moves[:10])

First 10 teoriat moves: 
[1, 0, 1, 0, 1, 0, 1, 0, 1, 0]


In [29]:
# turn white or black to integers ( 1  = (White), 0 = (Black) )
colors =[]
for color in df["moves"]:
    for c in color:
        if c[0] == "white":
            colors.append(1)
        else: #black
            colors.append(0)
# first 10 colors
print("First 10 colors: ")
print(colors[:10])

First 10 colors: 
[1, 0, 1, 0, 1, 0, 1, 0, 1, 0]


In [ ]:
#combine all the features into a single list of tuples 
Combined_features = []
for i in range(len(every_move)):
    Combined_features.append((
        colors[i],                 # integer for color
        move_to_number[every_move[i]],  # integer for move
        teoriat_moves[i]         # integer for teoriat moves
    ))

print("First 10 combined features: ")
print(Combined_features[:10])



First 10 combined features: 
[(1, 265, 1), (0, 1191, 0), (1, 1198, 1), (0, 127, 0), (1, 1118, 1), (0, 1078, 0), (1, 455, 1), (0, 213, 0), (1, 1706, 1), (0, 1101, 0)]
